In [81]:
#pip install mysql-connector-python

In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as po
import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import plotly
import plotly.graph_objs as go
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')
# Create a trace
data = [go.Scatter(
    x = df['data'],
    y = df['totale_positivi'],
)]
layout = go.Layout(
        xaxis=dict(
            title='Data',    
        ),
        yaxis=dict(
            title='Totale positivi',  
        )
    )
fig = go.Figure(data=data, layout=layout)

In [3]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [4]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [5]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [6]:
c = create_db_connection("giniewicz.it","team06","te@m0g","team06")

MySQL Database connection successful


In [7]:
#suma zarobionych pieniędzy ze sprzedaży gier w styczniu 2022
q1 = """
SELECT sum(game_purchase.price)
FROM game_purchase
WHERE DATE(game_purchase.purchase_date) BETWEEN '2022-01-01' AND '2022-01-31'
"""

#suma zarobionych pieniędzy z wypożyczeń gier w styczniu 2022
q2 = """
SELECT sum(game_rental.price)
FROM game_rental
WHERE DATE(game_rental.rental_date) BETWEEN '2022-01-01' AND '2022-01-31'
"""

# suma zarobków pracowników
q3 = """
SELECT sum(salary)
FROM employees
"""

#pracownicy
q4 = """
SELECT *
FROM employees
"""

#za ile pieniędzy w danym miesiącu pracownik sprzedał gier
q5 = """
SELECT sum(price) as t, employee_id, month(purchase_date) as m, year(purchase_date) as y, 
concat(employee_id,month(purchase_date),year(purchase_date)) as id
FROM game_purchase
GROUP BY y, m, employee_id
"""

#za ile pieniędzy w danym miesiącu pracownik wypożyczył gier
q6 = """
SELECT sum(game_rental.price) as t, game_rental.employee_id, month(game_rental.rental_date) as m, year(game_rental.rental_date) as y, 
concat(game_rental.employee_id,month(game_rental.rental_date),year(game_rental.rental_date)) as id,
concat(employees.first_name," ",employees.last_name) as names
FROM game_rental INNER JOIN employees on game_rental.employee_id = employees.employee_id
GROUP BY y, m, game_rental.employee_id
"""

#przychody ze sprzedaży
q7 = """
SELECT sum(game_purchase.price), shop_inventory.game_title
FROM game_purchase INNER JOIN shop_inventory ON game_purchase.game_id = shop_inventory.game_id
GROUP BY game_purchase.game_id
ORDER BY sum(game_purchase.price) DESC
LIMIT 5;
"""

#przychody z wypożyczeń
q8 = """
SELECT sum(game_rental.price), shop_inventory.game_title
FROM game_rental INNER JOIN shop_inventory ON game_rental.game_id = shop_inventory.game_id
GROUP BY game_rental.game_id
ORDER BY sum(game_rental.price) DESC
LIMIT 5;
"""

#top 10 zawodników szachów (inna gra - inne game_id)
q9 = """
SELECT t.total_score, CONCAT(k.first_name, ' ', k.last_name) AS full_name
FROM (
  SELECT SUM(score) AS total_score, customer_id
  FROM tournaments__scores
  INNER JOIN game_tournaments ON tournaments__scores.tournament_id = game_tournaments.tournament_id
  WHERE game_id = 1
  GROUP BY customer_id
  ORDER BY total_score DESC
  LIMIT 10
) AS t
INNER JOIN customers AS k ON t.customer_id = k.customer_id;
"""
# 5 klientów, którzy wydali w naszych sklepach najwięcej i ich wydatek podzielony przez ilosc dni miedzy pierwszym a ostatnim zakupem
q10 = """
SELECT (count(gr.game_id)+ count(gp.game_id))/(max(datediff(greatest(rental_date,purchase_date), least(rental_date,purchase_date)))+30) d,
concat(first_name," ",last_name) `imię i nazwisko`
FROM customers c
INNER JOIN game_purchase gp ON c.customer_id = gp.customer_id INNER JOIN game_rental gr ON gr.customer_id = c.customer_id
GROUP BY c.customer_id
ORDER BY d DESC
LIMIT 5
"""

q11 = """
SELECT concat(c.first_name," ", c.last_name), si.game_title, DATEDIFF(MAX(gr.return_date), MIN(gr.rental_date)) AS days_rented
FROM customers c
INNER JOIN game_rental gr ON c.customer_id = gr.customer_id
INNER JOIN shop_inventory si ON gr.game_id = si.game_id
GROUP BY c.customer_id,si.game_id
ORDER BY days_rented DESC
LIMIT 10;
"""

q12 = """
SELECT rental_date, COUNT(rental_id) AS count
FROM game_rental
GROUP BY rental_date
ORDER BY rental_date
"""

In [8]:
r1 = float(read_query(c,q1)[0][0])
r2 = float(read_query(c,q2)[0][0])
r3 = float(read_query(c,q3)[0][0])
r4 = read_query(c,q4)
r5 = read_query(c,q5)
r6 = read_query(c,q6)
r7 = read_query(c,q7)
r8 = read_query(c,q8)
r9 = read_query(c,q9)
r10 = read_query(c, q10)
r11 = read_query(c, q11)
r12 = read_query(c, q12)

In [9]:
df1 = pd.DataFrame({
    "przychód ze sprzedaży": [r1],
    "przychód z wypożyczeń": [r2],
    "koszt utrzymania pracowników": [r3]})
df4 = pd.DataFrame(r4, columns =['ID pracownika', "imię", "nazwisko", 'wypłata'])
df5 = pd.DataFrame(r5, columns =['cena kupna gry', 'ID pracownika', 'miesiąc', 'rok',"ID"])
df6 = pd.DataFrame(r6, columns =['cena wypożyczenia gry', 'ID pracownika', 'miesiąc', 'rok',"ID","imię i nazwisko"])
#df6_1 = df6
df6_1 = pd.merge(df5, df6, on='ID')
df6_1['miesiąc/rok'] = df6_1.apply(lambda x: f"{x['miesiąc_y']:02d}/{x['rok_y']}", axis=1)
df6_1["cena wypożyczenia gry"] = df6_1["cena wypożyczenia gry"].fillna(0)
df6_1["pełny przychód"] =( df6_1["cena wypożyczenia gry"] + df6_1["cena kupna gry"]).astype(int)
#df6_1 = df6.join(df5.set_index('ID pracownika'), on='ID')
df7 = pd.DataFrame(r7, columns =["przychód ze sprzedaży","nazwa gry"])
df8 = pd.DataFrame(r8, columns =["przychód z wypożyczeń","nazwa gry"])
df9 = pd.DataFrame(r9, columns =["całkowita punktacja", "imię i nazwisko"])
df10 = pd.DataFrame(r10, columns = ["znormalizowany wydatek","imię i nazwisko"])
df11 = pd.DataFrame(r11, columns = ["imię i nazwisko","tytuł gry", "dni wypożyczenia"])
df12 = pd.DataFrame(r12, columns = ["data", "ilość wypożyczeń"])

In [10]:
# zysk za styczeń 2022
s = r1+r2-r3

# top pracownicy
max_value_idx = df6_1.groupby(['rok_y','miesiąc_y'])['pełny przychód'].idxmax()
max_value_employees = df6_1.loc[max_value_idx][["imię i nazwisko","pełny przychód","miesiąc/rok"]].rename(columns={"name": "top_employee"})
top_emp = max_value_employees[["miesiąc/rok","imię i nazwisko"]][:-1].transpose()

In [11]:
employees = df6_1['imię i nazwisko'].unique()

fig = go.Figure()

for employee in employees:
    employee_df = df6_1[df6_1['imię i nazwisko'] == employee]
    fig.add_trace(go.Scatter(x=employee_df['miesiąc/rok'], y=employee_df['pełny przychód'], mode='lines+markers', name=employee))

max_scores = df6_1.groupby('miesiąc/rok')['pełny przychód'].max().reset_index()

fig.update_layout(xaxis_title='miesiąc/rok', yaxis_title='pełny przychód', title='Miesięczne wyniki sprzedaży każdego z pracowników')
p1 = po.plot(fig, output_type='div')

In [12]:
fig = go.Figure(data=go.Table(
    header=dict(values=[]),
    cells=dict(values=[top_emp[col] for col in top_emp.columns])
))
fig.update_layout(
    title='Pracownicy miesiąca'
)
p1_1 = po.plot(fig, output_type='div')

In [13]:
# top 10 zawodników
fig = px.bar(df9, x="imię i nazwisko", y="całkowita punktacja")
fig.update_layout(
    title="Punktacja w turnieju szachowym",
    xaxis_title="Imię i nazwisko",
    yaxis_title="Całkowita punktacja",
    xaxis_tickangle=-45,
)

p2 = po.plot(fig, output_type='div')

In [14]:
fig = make_subplots(rows=1, cols=2)

# "Całkowity przychód ze sprzedaży"
fig.add_trace(go.Bar(x=df7["nazwa gry"], y=df7["przychód ze sprzedaży"]), row=1, col=1)
fig.update_xaxes(title_text="Nazwa gry", tickangle=-45, row=1, col=1)
fig.update_yaxes(title_text="Przychód ze sprzedaży", row=1, col=1)

#"Całkowity przychód z wypożyczeń"
fig.add_trace(go.Bar(x=df8["nazwa gry"], y=df8["przychód z wypożyczeń"]), row=1, col=2)
fig.update_xaxes(title_text="Nazwa gry", tickangle=-45, row=1, col=2)
fig.update_yaxes(title_text="Przychód z wypożyczeń", row=1, col=2)

fig.update_layout(
    title="Największe przychody uzyskane w zależności od sprzedaży i wypożyczeń",
    showlegend=False,
)
p3 = po.plot(fig, output_type='div')

In [15]:
trace = go.Pie(
    labels=df10["imię i nazwisko"],
    values=df10["znormalizowany wydatek"]/sum(df10['znormalizowany wydatek']),
    hoverinfo='label+percent',
    textinfo='value',
    textfont=dict(size=15),
    marker=dict(colors=['#FFA07A', '#FFD700', '#7FFFD4', '#FF6347'])
)
layout = go.Layout(
    title='Najwierniejsi klienci'
)
fig = go.Figure(data=[trace], layout=layout)

p4 = po.plot(fig, output_type='div')

In [16]:
trace = go.Bar(
    x=df11['imię i nazwisko'],
    y=df11['dni wypożyczenia'],
    text=df11['tytuł gry'],
)

layout = go.Layout(
    title='Ilość dni wypożyczenia',
    xaxis=dict(title='Imię i nazwisko'),
    yaxis=dict(title='Ilość dni wypożyczenia',range=[520, 660])
)

fig = go.Figure(data=[trace], layout=layout)

p5 = po.plot(fig, output_type='div')

In [17]:
trace = go.Scatter(
    x=df12['data'],
    y=df12['ilość wypożyczeń'],
    mode='lines+markers',
    name='ilość' )

layout = go.Layout(
    title='Ilość wypożyczeń na przestrzeni czasu',
    xaxis=dict(title='Data'),
    yaxis=dict(title='Ilość wypożyczeń'))

figure = go.Figure(data=[trace], layout=layout)

top_ten = df12.sort_values('ilość wypożyczeń', ascending=False).head(10)
top_ten_trace = go.Scatter(
    x=top_ten['data'],
    y=top_ten['ilość wypożyczeń'],
    mode='markers',
    marker=dict(symbol='circle', size=10),
    name='Top 10 największej ilości wypożyczonych gier' 
)

trace.update(mode='lines')
figure.add_trace(top_ten_trace)

p6 = po.plot(figure, output_type='div')

In [18]:
html = f'''
<html>
    <head>
        <title>Wypożyczalnia gier planszowych</title>
        <style>
         table {{
      border-collapse: collapse;
    }}

    td {{
      text-align: center;
      vertical-align: middle;
    }}
    
    .col {{
  display: inline-block;
}}
        
.table-container {{
            width: 50%;
            margin: 0 auto;
        }}
        .table {{
  width: 100%;
  border-collapse: collapse;
}}

.table th,
.table td {{
  padding: 10px;
  border: 1px solid #ccc;
}}

.table th {{
  background-color: #f2f2f2;
  color: #333;
  font-weight: bold;
}}

.table tr:nth-child(even) {{
  background-color: #f9f9f9;
}}

.table tr:first-child {{
  font-weight: bold;
}}


            .highlight {{
                font-weight: bold;
                color: green;
            }}
        </style>
    </head>
    <body>
        <h1>Raport - analiza danych.</h1>
        Baza danych, która powstała zawiera 7 tabel (customers, employees, game_purchase,
        game_rental, game_tournaments, shop_inventory, tournaments_scores).
        Poniżej przedstawiamy analizę danych w niej zawartych. <br>
        Zakładamy, że dzień dzisiejszy to 14.02.2022.
        <h2>1. Pracownicy miesiąca</h2>
        <p>Na poniższym wykresie widzimy jaki dochód dla sklepu zyskał dany pracownik w kolejnych miesiącach</p>
        {p1}
        <p><br>Na podstawie zebranych danych wyznaczamy pracowników miesiąca (wykluczamy obecny, jescze nieskończony):</p>
        {top_emp.to_html(classes = 'table table-stripped col', col_space = "auto", justify = "center", header =False )}
        
        <h2>2. Top 10 zawodników</h2>
        <p>Punktacja najlepszych zawodników w turnieju szachowym:</p>
        {p2}
        
        <h2>3. Przychody w zależności od sprzedaży i wypożyczeń</h2>
        <p>Ustalamy, które gry przynoszą największy dochód ze sprzedaży, a które z wypożyczeń:</p>
        {p3}
        
        <h2>4. Własne</h2>
        <h3>4.1 Zyski za styczeń 2022</h3>
        {df1.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
        <p>Zyski z działalności wypożyczalni gier w styczniu 2022,
        obliczone jako różnica między sumą z przychodów ze sprzedaży i wypożyczeń gier,
        a sumą miesięcznych pensji pracowników:</p>
        <p><span class="highlight">{s} zł</span></p>
        
        <h3>4.2 Najwierniejsi klienci</h3>
        <p>Ustalamy pięciu najwierniejszych klientów przez zliczenie ilości wypożyczonych lub kupionych gier
        i podzielenie tej liczby przez ilość dni od momentu pierwszego wypożyczenia bądź zakupu do dnia za 30 dni.</p>
        {df10.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
        <p>Poniżej przedstawiamy stosunek otrzymanych wartości:</p>
        {p4}
        
        <h3> 4.3 Osoby z najdłuższym okresem wypożyczenia</h3>
        <p>Wykres przedstawia osoby które najdłużej przetrzymywały gry wraz z nazwą wypożyczonej gry.</p>
        {p5}
        
        <h3> 4.4 Ilość wypożyczonych gier na przestrzeni czasu</h3>
        <p>Wykres pokazuje jak na przestrzeni czasu wyglądała dzienna ilość sprzedanych i wypożyczon</p>
        {p6}
    </body>
</html>
    '''

with open('html_report.html', 'w') as f:
    f.write(html)